In [1]:
import syft as sy
import torch

In [2]:
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_root_client()
remote_torch = alice_client.torch

In [4]:
print(alice)
print(alice_client)
print(alice_client.vm_id)

VirtualMachine: alice: <UID: 6bea5357b3504c5a887396bc2d888a0c>
<VirtualMachineClient: alice Client>
<UID: 6bea5357b3504c5a887396bc2d888a0c>


In [5]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.fc1 = self.torch_ref.nn.Linear(100, 10)

    def forward(self, x):
        return self.fc1(x)

In [6]:
model = SyNet(torch)
data = torch.randn(size=(1, 100))
data

tensor([[-1.8305, -1.2449,  1.2339, -0.2621, -0.4630,  1.0017,  1.0102, -0.9429,
         -0.2143,  1.7321,  0.9425,  1.2148,  0.7220,  1.3547, -1.0784,  1.3045,
          0.0585,  1.7062,  2.0490,  0.2899, -1.2772,  0.8567,  0.6722, -1.9087,
          1.2654,  0.4542, -1.2053,  0.5683,  1.8294,  0.6104,  0.4707, -0.8326,
         -0.4360,  0.3185,  0.6080,  0.3666, -1.5240,  0.9798,  0.3765, -0.6078,
          1.0767, -1.1264, -0.5720,  0.8702, -1.4933, -0.4355,  0.3125,  0.1929,
          1.4774, -1.8456,  1.0463, -0.3143,  0.7398, -0.7127, -1.2736, -0.2679,
          1.1277, -0.7886, -0.2496, -0.3317, -0.0591,  0.7968,  0.6486,  0.5010,
         -0.6041, -1.2401, -0.8593,  1.1023, -0.3403, -1.1183,  0.0237, -0.1360,
          0.8747,  0.9837, -0.4689, -0.6797, -0.6207, -0.9985, -0.7924, -0.2051,
         -0.4115, -0.3115,  0.9180,  0.4193,  0.0475,  1.5892, -0.0089, -1.5990,
         -0.2699,  0.5302, -1.6310,  0.1887,  1.6549,  0.5280, -0.0239,  1.8766,
         -0.5467,  0.7146, -

In [7]:
result = model(data)
print(result)
labels = torch.randn(size=(1, 10))
print(labels)

tensor([[ 0.6644, -0.8585,  0.4058, -1.4926,  0.2167, -0.9398,  0.6130, -0.2456,
          0.2568,  0.6423]], grad_fn=<AddmmBackward>)
tensor([[ 0.1355,  0.1399, -0.6309,  1.7878, -0.4882,  0.2649, -0.8348, -1.2962,
          0.0381,  0.5024]])


In [8]:
loss_func = torch.nn.L1Loss()
loss = loss_func(result, labels)
loss.backward()

In [9]:
print(model.parameters()[-1].grad) # exists

model_ptr = model.send(alice_client)
data_ptr = data.send(alice_client)
labels_ptr = labels.send(alice_client)
results_ptr = model_ptr(data_ptr)
remote_loss_func = alice_client.torch.nn.L1Loss()
remote_loss = remote_loss_func(results_ptr, labels_ptr)
remote_loss.backward()

tensor([ 0.1000, -0.1000,  0.1000, -0.1000,  0.1000, -0.1000,  0.1000,  0.1000,
         0.1000,  0.1000])


In [10]:
print(model_ptr.parameters().get()[-1].grad) # exists
print(model_ptr.get().parameters()[-1].grad) # does not exist anymore

tensor([ 0.1000, -0.1000,  0.1000, -0.1000,  0.1000, -0.1000,  0.1000,  0.1000,
         0.1000,  0.1000])
None
